In [7]:
import ipydeps as ipd
modules = ["langchain", "pydantic", "discord.py"]

ipd.pip(modules)

In [ ]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.tools import tool
from pydantic import BaseModel
import discord
from discord import app_commands
import os
import json
import asyncio

In [10]:
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


@tool
def search_tool(query: str) -> str:
    """A tool to search for contact information in a database."""
    return f"Results for: {query}"

In [11]:
agent = create_agent(
    model = "gpt-4o",
    tools=[search_tool],
    response_format=ToolStrategy(ContactInfo)
)


# result = agent.invoke({"messages": [{"role": "user", "content": "Find me the contact info for John Doe, john@example.com, (555) 123 4567"}]})

# print(result["structured_response"])

In [ ]:



intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

@bot.event
async def on_ready():
    print(f"✅ Logged in as {bot.user}")


@bot.command()
async def user_info(ctx, *, message: str):
    """User command: !user_info <message>"""
    await ctx.send("Retrieving user info...")
    result = agent.invoke({
        "messages": [{"role": "user", "content": message}]
    })
    structured = result["structured_response"]
    await ctx.send(f"📄 **Agent Result:**\n```{structured}```")


# Run bot
bot.run(DISCORD_TOKEN)